## Datasets included
- "data_for_viz/consolidation-etalab-irve.csv": comporte les données des IRVE (https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/)

Dans le but de constituer un répertoire national des Infrastructures de recharge pour véhicules électriques (IRVE), ouvert et accessible à tous, les collectivités locales porteuses d’un projet d’installation d’IRVE doivent, au fur et à mesure de la mise en service des stations, publier sur la plateforme data.gouv.fr les données statiques relatives à la localisation et aux caractéristiques techniques de ces installations selon les modalités définies dans l’arrêté du 4 mai 2021.
Etalab consolide l'ensemble des jeux de données produits par les différents acteurs territoriaux sur un jeu de donnée consolidé. Celui-ci a pour objectif d'être le plus exhaustif possible et ambitionne de regrouper l'ensemble des bornes IRVE françaises. Un document décrivant l'ensemble des datasources utilisées pour cette consolidation peut être consulté sur la page data.gouv de la datasource.

[Datasource data.gouv](https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/)

- "data_for_viz/voitures-rechargeables-par-commune-enrichies.csv": comporte le nombre de voitures immatriculées par commune et par type de charge (https://www.data.gouv.fr/fr/datasets/voitures-particulieres-immatriculees-par-commune-et-par-type-de-recharge/), données enrichies avec le département (https://geo.api.gouv.fr/decoupage-administratif/communes).

- "data_for_viz/communes-departement-region.csv": dataset comportant les communes de France, avec pour chacune leur département et région

- "data_for_viz/departements.geojson": comporte la géométrie des départements (https://france-geojson.gregoiredavid.fr/)

In [1]:
# Set up environment

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
#import plotly.figure_factory as ff
import seaborn as sns

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## 1. Clean IRVE data
### 1.A Clean variables

In [2]:
irve = pd.read_csv(r"path\consolidation-etalab-irve.csv", sep=',', on_bad_lines='skip', dtype=str)

# Clean boolean variables
map_to_bool = {
    'false': False,
    '0': False,
    'FALSE': False,
    'False': False,
    'true': True,
    'TRUE': True,
    '1': True,
    'True': True
}

bool_columns = ['prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
               'gratuit', 'paiement_acte', 'paiement_cb', 'paiement_autre', 'reservation', 'station_deux_roues',
                'consolidated_is_lon_lat_correct', 'consolidated_is_code_insee_verified']

for col in bool_columns:
    irve[col] = irve[col].map(map_to_bool)

# Check
for col in bool_columns:
    print(col, irve[col].unique())


# Assign other datatypes
types_dict = {
    'nbre_pdc': int, 
    'puissance_nominale': float, 
    'consolidated_longitude': float,
    'consolidated_latitude': float,
              }

for col, col_type in types_dict.items():
    irve[col] = irve[col].astype(col_type)


prise_type_ef [False  True]
prise_type_2 [False  True]
prise_type_combo_ccs [ True False]
prise_type_chademo [False  True]
prise_type_autre [False  True]
gratuit [False True nan]
paiement_acte [ True False]
paiement_cb [True False nan]
paiement_autre [True False nan]
reservation [ True False]
station_deux_roues [False  True]
consolidated_is_lon_lat_correct [ True False]
consolidated_is_code_insee_verified [ True False]


### 1.B Treat Duplicates
- Based on what follow, we retain "point de charge" as the way to identify a unique charging station.

**From Nalron:**
Notons que le contexte métier nécessite de la rigueur dans l'interprétation de certaines variables, l'amalgame entre station, borne et point de charge est régulièrement rencontré. Donc, "id_station" n'est pas le sous-ensemble le plus approprié à l'identification de doublons, une station de recharge peut avoir plusieurs points de charge, et l'identifiant ne tient pas compte du point de charge. Notons que "id_pdc_itinerance" permet d'obtenir des identifiants uniques pouvant cette fois-ci être pris comme sous-ensemble. 

**Combien de points de charge en France?**
Selon la définition de l'AFIREV, le point de charge représente le nombre d'emplacements individuels permettant le stationnement du véhicule pendant le temps de charge, donc le nombre de prises de la borne. Le jeu de données `irve` ne permet pas de le quantifier directement, malgré la présence d'une variable 'nbre_pdc' qui ne représente que la borne et non le nombre de prises.

Les articles suivants permettent de se faire une idée de l'évolution du nombre de points de charge en France en 2022 et 2023 : 

**Article de Février 2023:** [Source](https://www.tressol-chabrier.com/actualites/Le+futur+de+l%E2%80%99automobile/Voiture%2B%25C3%25A9lectrique%2B%253A%2B85%2B284%2Bpoints%2Bde%2Brecharge%2Bdisponibles%2Ben%2BFrance-107) 

"85 284, c’est le nombre de points de recharge ouverts au public au 31 janvier 2023. Il est donc aujourd’hui de plus en plus facile de recharger sa voiture électrique. Il s’agit d’une évolution de + 57 % en un an. Si ces efforts se poursuivent, l’objectif pour la France d’atteindre les 100 000 points de recharge est ainsi facilement atteignable."


**Article de Avril 2022:** [Source](https://www.lesnumeriques.com/voiture/le-chiffre-du-jour-57-732-bornes-de-recharge-pour-voitures-electriques-en-france-n180505.html) 

"Il indique que la France peut désormais compter sur 57 732 points de recharge ouverts au public au 31 mars 2022. L'Avere France se satisfait d'un taux d'évolution de 54 % sur 12 mois"

In [3]:
# Count duplicates for different possible identifiers
identifiers = ['id_station_itinerance', 'id_station_local', 'id_pdc_itinerance', 'id_pdc_local']
for col in identifiers:
    duplicate_sum=irve[col].duplicated().sum()
    n_unique=irve[col].nunique()
    print(col, duplicate_sum, n_unique)

#How many stations in March 2023
irve.id_station_itinerance.nunique()

#How many charging stations in March 2023.
irve.id_pdc_itinerance.nunique()

# 43589 charging stations but still 11731 duplicates. What to do with them?
irve_duplicates = irve[irve['id_pdc_itinerance'].duplicated() == True]
print(irve_duplicates.groupby('id_pdc_itinerance'))
print(irve_duplicates.duplicated().sum())

# Among duplicated id_pdc_itinerance, all of them differ by at least one variable --> Keep all of them.

id_station_itinerance 36594 18726
id_station_local 44396 10923
id_pdc_itinerance 11731 43589
id_pdc_local 27253 28066
0


### 1.C Deal with missing department/ municipalities data
- First we clean the department and municipalities data and merge them with irve
- For the 30% irve observations with missing department/ municipalities we input them based on XY coordinates


In [4]:
# Read department data
geojson_dep = gpd.read_file(r"path\departements.geojson")

In [5]:
# Read municipalities data
df_communes=pd.read_csv(r"path\communes-departement-region.csv", dtype=str)

# Clean municipalities data

# Clean INSEE municipalities codes (add first missing '0')
df_communes.loc[df_communes['code_commune_INSEE'].str.len() == 4, "code_commune_INSEE"] = '0' + df_communes['code_commune_INSEE']
df_communes.loc[df_communes['code_postal'].str.len() == 4, 'code_postal'] = '0' + df_communes['code_postal']
df_communes.loc[df_communes['code_departement'].str.len() == 1, 'code_departement'] = '0' + df_communes['code_departement']
df_communes = df_communes[['code_commune_INSEE', 'code_departement', 'nom_departement']]

# Drop duplicated INSEE codes
df_communes = df_communes.drop_duplicates(subset='code_commune_INSEE')

In [6]:
# Merge IRVE with municipalities data
irve = irve.merge(df_communes, left_on='code_insee_commune', right_on='code_commune_INSEE', how='left')

# Count department code NaN  
irve['code_departement'].value_counts(ascending=True, dropna=False, normalize=True) #32% observations are NaN

# Transform lat/long couple into geometric point
irve['point'] = gpd.GeoSeries.from_xy(irve['consolidated_longitude'], 
                                             irve['consolidated_latitude'])
irve = gpd.GeoDataFrame(irve, geometry='point')

# Put department code as index in geojson database
geojson_dep = geojson_dep.set_index('code')

# select stations with missing department 
missing_dep = irve[irve['code_departement'].isna()]

# Convert 'point' columns into "Point" Shapely object
missing_dep['geometry'] = missing_dep['point'].apply(Point)

# Look for department based on whether the department polygon includes the point
# For each station with missing department: 
for i, point in missing_dep.iterrows():
    # trouver l'index (code du département) des polygones du DataFrame geopandas des departments qui comportent le point
    polygons_containing_point = geojson_dep[geojson_dep['geometry'].contains(point['geometry'])].index.tolist()
    # Ajouter le code du département trouvé (si un département a été trouvé)
    for polygon_index in polygons_containing_point:
        irve.loc[i, 'code_departement'] = polygon_index
        break

# convert from gpd DataFrame format to Dataframe
irve=pd.DataFrame(irve)

c:\Users\kzf401\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### 1.D Additional stuff

In [7]:
# For now we leave out the cleaning of NaN for n_amenageur, n_operateur, and n_enseigne done by Nalron

# Add info (population, population density and region) at the department level based on 'departements-francais.csv'
#Source : https://www.regions-et-departements.fr/departements-francais

dpt_fr = pd.read_excel(r"path\departements-francais.xls")

# Clean department data
dpt_fr.rename(columns={'NUMÉRO': 'code_dpt', 'NOM': 'dpt', 'REGION': 'region',
                       'SUPERFICIE (km²)': 'superficie_km2', 'POPULATION': 'nbre_habitants',
                       'DENSITE (habitants/km2)': 'hab/km2'}, inplace=True)

dpt_fr['code_dpt'] = dpt_fr['code_dpt'].astype('str')
dpt_fr.loc[dpt_fr['code_dpt'].str.len() == 1, 'code_dpt'] = '0' + dpt_fr['code_dpt']

# Merge irve and additional info on departments
irve = irve.merge(dpt_fr[['code_dpt', 'region', 'superficie_km2', 'nbre_habitants', 'hab/km2']], 
                    how='left', left_on="code_departement", right_on = "code_dpt")

#### Export cleaned irve dataset

In [8]:
# Export clean data
irve.to_csv(r'path\consolidation-etalab-irve-clean.csv')

## 2. Create Irve-based datasets for data viz
### 2.A Location-level data for maps

In [9]:
## Viz: map irve/ department

# Group by location
pdc_par_loc = irve.groupby(['consolidated_longitude', 'consolidated_latitude']).agg({
    'nbre_pdc': 'count',
    'code_departement': lambda x: pd.Series.mode(x)
}).reset_index()

# Re-create point
pdc_par_loc['point']= gpd.GeoSeries.from_xy(pdc_par_loc['consolidated_longitude'], 
                                             pdc_par_loc['consolidated_latitude'])

# Replace [] with NaN
def find_dep(dep):
    if isinstance(dep, str):
        return dep
    return np.nan

pdc_par_loc['code_departement'] = pdc_par_loc['code_departement'].apply(find_dep)

# Keep only metropolitan France for viz
pdc_par_loc = pdc_par_loc[pdc_par_loc['code_departement'].astype(str).apply(len)<3]

# Export file for data viz
pdc_par_loc.to_csv(r'path\irve_par_loc.csv', index=False)

### 2.B Time series data for evolutions

In [10]:
# Créer un timeseries du nombre de stations de recharge au cours du temps

# remplacer les dates aberrantes par des valeurs nulles
irve.loc[(irve['date_mise_en_service']>'2024') | (irve['date_mise_en_service']<'2001'), 
         'date_mise_en_service'] = np.nan

# convertir la colonne en datetime
irve['date_mise_en_service'] = pd.to_datetime(irve['date_mise_en_service'], format="%Y-%m-%d")

# grouper par mois, appliquer une somme cumulative sur le nombre de stations et de points de charge
irve.set_index('date_mise_en_service', inplace=True)
irve_ts = (irve.groupby(pd.Grouper(freq="M")).nbre_pdc
               .agg(['count', 'sum'])
               .add_prefix('nbre_pdc_')
               .cumsum()
               .reset_index())

irve_ts = (irve.groupby(pd.Grouper(freq="M")).agg({
    'id_station_itinerance': 'nunique',
    'id_pdc_itinerance': 'nunique'
}).cumsum().reset_index()).rename(columns={'id_station_itinerance': 'nb_stations',
                                           'id_pdc_itinerance': 'nb_pdc'})

# exporter le fichier
irve_ts.to_csv(r'path\irve_time_series.csv')